# Create an InterActive Graph of NTRL Lab Analysis Results

In [5]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
#from sklearn.metrics import mean_squared_error
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis

In [75]:
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time

def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------
class LabResults():
    def __init__(self, monthyear, filePath):
        #____monthyear format examle is datetime.date(2020,7,1)
        #____filepath example is '\\thgobna001\userdata\THPAL\...\ANALYSIS RESULTS 2020\7) July _2020.xlsx'
        #____add r' at the start of the filepath to apply the regex that allows the use of the filepath as raw
        #____sef.workbook is the uploaded excel workbook of the entire analysis files so it takes time to load
        #____Create an instance of the workbook by giving it a varible name with the sample format: Jan_2020
        
        import openpyxl
        import io
        with open(filePath, "rb") as f:
            in_mem_file = io.BytesIO(f.read())
        
        self.workbook = openpyxl.load_workbook(in_mem_file, data_only=True)
        self.monthyear = monthyear   
        
    def generate_df_from_analysis(self, tabname, list_lab_index):
        #____tabname is a string type name of the tab in the laboratory analysis excel file.
        #____-------Example tabnames: '104PU01', '105TK03', '106TK01'
        #____-------Inside these tabs are the specific analysis results done on the sample taken from the 'tabname' area .
        #____list_lab_index is a list with the sample format is: ['B', 'C', 'E', '104_Pb']
        #____-------the first letter 'B' is the column letter where the day of the month is written as integers 1,2,3..30
        #____-------the second letter 'C' is the column letter where the sampling time is written; e.g. '7:00:00 AM'
        #____-------the third letter 'E' is the column where the results of the specific analysis are listed
        #____-------the last item in the list is the string you'll use as the new title; e.g. '104_Pb'
        
        workbookTab = self.workbook[tabname]
        self.list_lab_index = list_lab_index
        col_sampleDate = self.list_lab_index[0]
        col_sampleTime = self.list_lab_index[1]
        col_analysis = self.list_lab_index[2]
        col_title = self.list_lab_index[3]
        currentmonthyr = self.monthyear
        
        #get the start and end sample dates
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
        #gets the date day from the B column of the analysis file if there is value in the cell
        for i in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
                Smpl_day_index.append(i)
        #get the ending row that has date day
        for j in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
                smpl_end_index = j
        for y in range(0, len(Smpl_day_index)):
            if Smpl_day_index[y] != Smpl_day_index[-1]:      
                for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                    if workbookTab[str(col_analysis)+str(x)].value is None: 
                        pass
                    else:
                        if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                            date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))  
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)
                        else:
                            date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))  
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)

            if Smpl_day_index[y] == Smpl_day_index[-1]:
                for x in range(Smpl_day_index[y], smpl_end_index+1):
                    if workbookTab[str(col_analysis)+str(x)].value is None:   
                        pass
                    else:
                        if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                            date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)
                        else:
                            date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)

        Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
        Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
        Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
        Chem_Element_df.set_index('DATETIME', inplace=True)

        return Chem_Element_df
    
def join_DF(dfsList1):
    df = dfsList1[0]
    for i in range(1,len(dfsList1)):
        df = df.join(dfsList1[i], how='outer')
    return df

def append_monthsDF(dfsListmonthly): #takes the list of dataframes from monthly lab results of one sampling area
    df = dfsListmonthly[0]
    for i in range(1,len(dfsListmonthly)):
        df = df.append(dfsListmonthly[i], ignore_index=True)
    return df    
def generate_combined_df_by_month(month_WB, sample_area, analysis_type):
    df_List = []
    for i in analysis_type:
        df_analysis = month_WB.generate_df_from_analysis(sample_area, i)
        df_List.append(df_analysis)
    for i in df_List:
        df = join_DF(df_List)
    return df



#----------------------------------------------------End-----------------------------------------------------------------
#
#  make_df_for_each_FT01_by_month_then_export_excel: This function exports the 
def make_df_for_each_FT01_by_month(monthLabRes, analysis_type_FT01AtoF_list):
    #month_ = LabResults(currentmonthyr_list[monthNumber-1], filepath_list[monthNumber-1])
    letters = 'ABCDEF'
    FT01_DF_list_by_filter = []
    for i in range(0,6):
        FT01_ = generate_combined_df_by_month(monthLabRes, '105FT01', analysis_type_FT01AtoF_list[i])
        FT01_.columns = [letters[i]+'_Zn',
                         letters[i]+'_ntu',
                         letters[i]+'_Zn_dr']
        FT01_DF_list_by_filter.append(FT01_)
    return join_DF(FT01_DF_list_by_filter)    

#--------------------------------------------------End-------------------------------------------------------------------
#
#
def generate_plot_from_df(figname, df,df_axis_truefalse_list):
    for i in range(0, len(df.columns)):
        colname = df.columns[i]
        figname.add_trace(go.Scatter(x=df[colname].index, y=df[colname], name=colname), secondary_y=df_axis_truefalse_list[i],)
#
#
#--------------------------------------------------------------End-----------------------------------------------------------------------

In [35]:
yesterday = '07'
wb1 = openpyxl.load_workbook(r'\\THGOBNA002\thpal\05 Production\302 Management - Report\本部長報告資料\Daily Specific Demand control 2020\202008\Daily specific demand trend(2020_8_{}).xlsx'.format(yesterday), data_only=True)
#Drop the row[1]
#####This block of code appends the Analysis Results from 
Basic_unit = wb1['原単位']
Slakedlime_rxn_eff = wb1['消石灰反応効率']

#update for the targetmonth
currentmonthyr = datetime.date(2020,8,1)
print('Done', datetime.datetime.now())

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:296: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\reader\drawings.py:59: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)


Done 2020-08-08 14:33:09.770336


In [83]:

def get_Chem_con(workbook, index, index_row_start,  index_row_end, column, chem_row_start, chem_row_end, title):
    datelist = []
    Chemical_con = []
    for y in range(chem_row_start, chem_row_end+1):
        
        chem = workbook[str(column)+str(y)].value
        Chemical_con.append(chem)
    for i in range(index_row_start, index_row_end+1):
        
        Index_date = workbook[str(index)+str(i)].value
        datelist.append(parse(str(Index_date)))
        

    Chem_con_list_pair = list(zip(Chemical_con, datelist[0:]))    
    Chem_con_df = pd.DataFrame(Chem_con_list_pair)    

    Chem_con_df.columns = [title, 'DATETIME'] ##check if list title is updated
    Chem_con_df.set_index('DATETIME', inplace=True)    
    return Chem_con_df


def get_Chem_con_2WB(workbook1, workbook2, index, index_row_start,  index_row_end, column, chem_row_start, chem_row_end, title):
    datelist = []
    Chemical_con = []
    for y in range(chem_row_start, chem_row_end+1):
        
        chem = workbook2[str(column)+str(y)].value
        Chemical_con.append(chem)
    for i in range(index_row_start, index_row_end):
        
        Index_date = workbook1[str(index)+str(i)].value
        datelist.append(parse(str(Index_date)))
        

    Chem_con_list_pair = list(zip(Chemical_con, datelist[0:]))    
    Chem_con_df = pd.DataFrame(Chem_con_list_pair)    

    Chem_con_df.columns = [title, 'DATETIME'] ##check if list title is updated
    Chem_con_df.set_index('DATETIME', inplace=True)    
    return Chem_con_df

In [84]:
def join_DF(dfsList1):
    df = dfsList1[0]
    for i in range(1,len(dfsList1)):
        df = df.join(dfsList1[i], how='outer')
    return df

In [85]:
# DFs for Limestone SD actual consumption graph

wic014_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'AW', 9, 39, 'WIC014')
wic006_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'Q', 9, 39, 'WIC006')
flowratebase_df = get_Chem_con_2WB(Basic_unit, Slakedlime_rxn_eff, 'O', 9, 39, 'AK', 44, 74, 'Flowrate_base')
LS_SD_budget_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'C', 86, 115, 'LS_SD_budget')

LS_wic_df = join_DF([wic014_df, wic006_df, 
                     flowratebase_df, LS_SD_budget_df])

# DFs for Limestone SD area consumption graph
ls_pre_neut_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'R', 9, 39, 'PRE_NEUT')
ls_pre_coal_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'BE', 9, 39, 'coal')
ls_pre_Fneut_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'S', 9, 39, 'F_NEUT')


ls_budget_pre_neut_ntrl_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'D', 9, 39, 'LS_Budget_PRE_NEUT_NTRL')
ls_budget_Fneut_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'E', 9, 39, 'LS_Budget_FNTRL')

LS_per_areaUsage_df = join_DF([ls_pre_neut_df,
                               ls_pre_coal_df,
                               ls_pre_Fneut_df,
                               ls_budget_pre_neut_ntrl_df,
                               ls_budget_Fneut_df])

# DFs for Slakedlime SD graph
SL_dilRatio_Fneut_df = get_Chem_con(Slakedlime_rxn_eff, 'A', 44, 74, 'X', 44, 74, 'S/L_dil_Ratio')
SL_Actual_Fneut_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'V', 9, 39, 'S/L_actual_F_NEUT')
SL_Budget_Fneut_df = get_Chem_con(Basic_unit, 'B', 86, 116, 'G', 86, 116, 'S/L_budget_F_NEUT')

SL_fneut_df = join_DF([SL_dilRatio_Fneut_df,
                       SL_Actual_Fneut_df,
                       SL_Budget_Fneut_df])

In [86]:
# DFs for Slaked lime efficiency graph

SL_rxn_eff_Budget_df = get_Chem_con(Basic_unit, 'B', 86, 116, 'H', 86, 116, 'S/L_Rxn_eff_budget')
SL_SD_Mg_df = get_Chem_con(Slakedlime_rxn_eff, 'A', 44, 74, 'AC', 44, 74, 'S/L_SD_Mg')

SL_rxn_eff_df = get_Chem_con(Slakedlime_rxn_eff, 'A', 44, 74, 'B', 44, 74, 'S/L_rxn_eff')

SL_SD_others_df = get_Chem_con(Slakedlime_rxn_eff, 'A', 44, 74, 'AG', 44, 74, 'S/L_SD_others')
SL_SD_Mn_df = get_Chem_con(Slakedlime_rxn_eff, 'A', 44, 74, 'AG', 44, 74, 'S/L_SD_Mn')

SL_SD_Fe_df = get_Chem_con(Slakedlime_rxn_eff, 'A', 44, 74, 'AE', 44, 74, 'S/L_SD_Fe')

SL_rxn_eff_df = join_DF([SL_rxn_eff_Budget_df,
                         SL_SD_Mg_df,
                         SL_rxn_eff_df,
                         SL_SD_others_df,
                         SL_SD_Mn_df,
                         SL_SD_Fe_df])


In [87]:
# DFs for NaOH

NaOH_SD_act_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'AP', 9, 39, 'NaOH_SD_actual_consumption')
NaOH_SD_budget_df = get_Chem_con(Basic_unit, 'B', 86, 116, 'SS', 86, 116, 'NaOH_SD_budget')

NaSH_031_molpermol_df = get_Chem_con(Basic_unit, 'B', 86, 116, 'T', 86, 116, 'NaSH_0.31_molpermol')
NaOH_SD_MSrxtr_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'AS', 9, 39, 'NaOH_SD_MSrxtr')

NaOH_SD_MS_sr_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'AQ', 9, 39, 'NaOH_SD_MS_Scrubber')

NaOH_SD_H2S_sr_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'AR', 9, 39, 'NaOH_SD_H2S_Scrubber')

NaOH_SD_FNTRL_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'AT', 9, 39, 'NaOH_SD_FNTRL')

NaOH_SD_df = join_DF([NaOH_SD_act_df,
                      NaOH_SD_budget_df,
                      NaSH_031_molpermol_df,
                      NaOH_SD_MSrxtr_df,
                      NaOH_SD_MS_sr_df,
                      NaOH_SD_H2S_sr_df,
                      NaOH_SD_FNTRL_df])

# DFs for H2SO4

acid_SD_actHPAL_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'Y', 9, 39, 'H2SO4_HPAL_actual')
acid_SD_budgetHPAL_df = get_Chem_con(Basic_unit, 'B', 86, 116, 'K', 86, 116, 'H2SO4_SD_HPAL_budget')
acid_SD_MSbudget_df = get_Chem_con(Basic_unit, 'B', 86, 116, 'L', 86, 116, 'H2SO4_SD_MS_budget')
acid_SD_MSactual_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'AQ', 9, 39, 'H2SO4_SD_MS_actual')

acid_SD_df = join_DF([acid_SD_actHPAL_df,
                     acid_SD_budgetHPAL_df,
                     acid_SD_MSbudget_df,
                     acid_SD_MSactual_df])


In [88]:
# DFs for Ni throughput
# use a modified function for this one
# def get_Chem_con_2WB(workbook1, workbook2, index, index_row_start,  index_row_end, column, chem_row_start, chem_row_end, title):
Ni_throughput_df = get_Chem_con_2WB(Basic_unit, Slakedlime_rxn_eff, 'O', 9, 39, 'AL', 44, 74, 'Ni_throughput')
Ni_loss_DeZn_df = get_Chem_con_2WB(Basic_unit, Slakedlime_rxn_eff, 'O', 9, 39, 'AT', 44, 74, 'Ni_loss_DeZn')
Ni_loss_HPAL_df = get_Chem_con_2WB(Basic_unit, Slakedlime_rxn_eff, 'O', 9, 39, 'AR', 44, 74, 'Ni_loss_HPAL')
Ni_loss_CCD_df = get_Chem_con_2WB(Basic_unit, Slakedlime_rxn_eff, 'O', 9, 39, 'AS', 44, 74, 'Ni_loss_CCD')
Ni_loss_BL_df = get_Chem_con_2WB(Basic_unit, Slakedlime_rxn_eff, 'O', 9, 39, 'AU', 44, 74, 'Ni_loss_BL')
Ni_loss_TSS_df = get_Chem_con_2WB(Basic_unit, Slakedlime_rxn_eff, 'O', 9, 39, 'AV', 44, 74, 'Ni_loss_TSS')

Ni_throughput_loss_df = join_DF([Ni_throughput_df,
                                 Ni_loss_DeZn_df,
                                 Ni_loss_HPAL_df,
                                 Ni_loss_CCD_df,
                                 Ni_loss_BL_df,
                                 Ni_loss_TSS_df])

In [89]:
# DFs for Flocc graph

Flocc_total_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'AG', 9, 39, 'SD_flocc_total')
Floc_totalbudget_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'N', 86, 116, 'Floc_(total)_budget')
Floc_CCD_MS_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'AF', 9, 39, 'Floc_(CCD_MS)')
Kuri_per_PLS_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'BI', 47, 77, 'Kurifix_per_PLS_ppm')
Kurifix_SD_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'AI', 9, 39, 'Kurifix_SD')
Kurifix_SD_budget_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'Q', 86, 116, 'Kurifix_SD_budget')
Floc_CCD_MS_budget_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'P', 86, 116, 'Floc_(CCD_MS)_budget')
Floc_ORE_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'AD', 9, 39, 'Floc_(ORE)')
Floc_ORE_budget_df = get_Chem_con(Basic_unit, 'O', 9, 39, 'O', 86, 116, 'Floc_(ORE)_budget')

Flocc_SD_df = join_DF([Flocc_total_df,
                       Floc_totalbudget_df,
                       Floc_CCD_MS_df,
                       Kuri_per_PLS_df,
                       Kurifix_SD_df,
                       Kurifix_SD_budget_df,
                       Floc_CCD_MS_budget_df,
                       Floc_ORE_df,
                       Floc_ORE_budget_df])

In [82]:
Floc_totalbudget_df.dropna()

,Floc_(total)_budget
DATETIME,
2020-08-01 06:00:00,17.101613
2020-08-02 06:00:00,25.703413
2020-08-03 06:00:00,21.916395
2020-08-04 06:00:00,19.016256
2020-08-05 06:00:00,22.830729
2020-08-06 06:00:00,25.029704
2020-08-07 06:00:00,21.443355


In [123]:
print(Flocc_SD_df.columns)
print(len((Flocc_SD_df.columns)))

Index(['SD_flocc_total', 'Floc_(total)_budget', 'Floc_(CCD_MS)',
       'Kurifix_per_PLS_ppm', 'Kurifix_SD', 'Kurifix_SD_budget',
       'Floc_(CCD_MS)_budget', 'Floc_(ORE)', 'Floc_(ORE)_budget',
       'FlocTot_diff_budget'],
      dtype='object')
10


In [91]:
Flocc_SD_df.dropna()

,SD_flocc_total,Floc_(total)_budget,Floc_(CCD_MS),Kurifix_per_PLS_ppm,Kurifix_SD,Kurifix_SD_budget,Floc_(CCD_MS)_budget,Floc_(ORE),Floc_(ORE)_budget
DATETIME,,,,,,,,,
2020-08-01 06:00:00,0.050853,0.04019,0.033877,65.543,0.023545,0.019,0.016,0.016976,0.012
2020-08-02 06:00:00,0.048610,0.04019,0.030931,62.2817,0.023452,0.019,0.016,0.017679,0.012
2020-08-03 06:00:00,0.047554,0.04019,0.031165,59.4296,0.019572,0.019,0.016,0.016388,0.012
2020-08-04 06:00:00,0.050697,0.04019,0.035658,65.8334,0.022464,0.019,0.016,0.015040,0.012
2020-08-05 06:00:00,0.053096,0.04019,0.034477,68.8968,0.023112,0.019,0.016,0.018620,0.012
2020-08-06 06:00:00,0.060506,0.04019,0.039283,59.7194,0.020471,0.019,0.016,0.021224,0.012
2020-08-07 06:00:00,0.056968,0.04019,0.036692,59.7672,0.021823,0.019,0.016,0.020276,0.012


In [134]:
#Flocc_SD_df
#Ni_throughput_loss_df
#acid_SD_df
#NaOH_SD_df
#SL_rxn_eff_df
#LS_wic_df
#LS_per_areaUsage_df
#SL_fneut_df



#----------------------------------------------------------------------  NTRL----------------------------------------------------------------------------
#
#
figfloc = make_subplots(specs=[[{"secondary_y": True}]])
figfloc_truefalse_list = [False, False, False, True, False,
                               False, False, False, False, False]

generate_plot_from_df(figfloc, Flocc_SD_df.dropna(), figfloc_truefalse_list)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
t = datetime.datetime.now()
title_plotfloc = "<b>Flocc SD</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title
figfloc.update_layout(title_text=title_plotfloc,xaxis=dict(rangeslider=dict(visible = True),type='date'))
# Set x-axis title
figfloc.update_xaxes(title_text="DateTime")
# Set y-axes titles
figfloc.update_yaxes(title_text="<b> </b>", secondary_y=False)
figfloc.update_yaxes(title_text="<b> </b>", secondary_y=True)
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------
#figfloc.add_trace(go.Bar(x=Flocc_SD_df.index, y=Flocc_SD_df['error'], name='error'), secondary_y=True)
plt.close()



In [138]:
len(Ni_throughput_loss_df.columns)

6

In [139]:
#----------------------------------------------------------------------  NTRL----------------------------------------------------------------------------
#
#Ni_throughput_loss_df
figNi_throu = make_subplots(specs=[[{"secondary_y": True}]])
figNi_truefalse = [False, False, False, False, False,
                               False]

generate_plot_from_df(figNi_throu, Ni_throughput_loss_df.dropna(), figNi_truefalse)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
t = datetime.datetime.now()
title_plotNi = "<b>Ni through</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title
figNi_throu.update_layout(title_text=title_plotNi,xaxis=dict(rangeslider=dict(visible = True),type='date'))
# Set x-axis title
figNi_throu.update_xaxes(title_text="DateTime")
# Set y-axes titles
figNi_throu.update_yaxes(title_text="<b> </b>", secondary_y=False)
figNi_throu.update_yaxes(title_text="<b> </b>", secondary_y=True)
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------
#figfloc.add_trace(go.Bar(x=Flocc_SD_df.index, y=Flocc_SD_df['error'], name='error'), secondary_y=True)
plt.close()



In [140]:
figNi_throu

In [105]:
Flocc_SD_df['FlocTot_diff_budget'] = Flocc_SD_df['Floc_(total)_budget'] - Flocc_SD_df['SD_flocc_total']
Flocc_SD_df


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,SD_flocc_total,Floc_(total)_budget,Floc_(CCD_MS),Kurifix_per_PLS_ppm,Kurifix_SD,Kurifix_SD_budget,Floc_(CCD_MS)_budget,Floc_(ORE),Floc_(ORE)_budget,FlocTot_diff_budget
DATETIME,,,,,,,,,,
2020-08-01 06:00:00,0.050853,0.04019,0.033877,65.543,0.023545,0.019,0.016,0.016976,0.012,-0.010662
2020-08-02 06:00:00,0.048610,0.04019,0.030931,62.2817,0.023452,0.019,0.016,0.017679,0.012,-0.008420
2020-08-03 06:00:00,0.047554,0.04019,0.031165,59.4296,0.019572,0.019,0.016,0.016388,0.012,-0.007363
2020-08-04 06:00:00,0.050697,0.04019,0.035658,65.8334,0.022464,0.019,0.016,0.015040,0.012,-0.010507
2020-08-05 06:00:00,0.053096,0.04019,0.034477,68.8968,0.023112,0.019,0.016,0.018620,0.012,-0.012906
2020-08-06 06:00:00,0.060506,0.04019,0.039283,59.7194,0.020471,0.019,0.016,0.021224,0.012,-0.020316
2020-08-07 06:00:00,0.056968,0.04019,0.036692,59.7672,0.021823,0.019,0.016,0.020276,0.012,-0.016777


In [ ]:

plt.close()
with open(r'\\THGOBNA002\thpal\05 Production\150 DCS\Miscellaneous\InterActive Plots Specific_Demand.html', 'w') as f:
    f.write(figMSVE04_PRED.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figNTRL.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figDeZnTK03.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figDeZnTK05.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figMSTK01.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figMSVE04.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figPolishing.to_html(full_html=False, include_plotlyjs='cdn'))
   

